# Supervised NLP (The King James Bible vs. Shakepseare's)
+ pre-labelled dataset

In [184]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter
import nltk
import warnings
warnings.simplefilter('ignore')

In [185]:
nltk.corpus.gutenberg.fileids()

['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

In [186]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
bible = gutenberg.raw('bible-kjv.txt')
shakespeare = gutenberg.raw('shakespeare-caesar.txt')

# The Chapter indicator is idiosyncratic
bible = re.sub(r'Chapter \d+', '', bible)
bible = re.sub(r'\d+:\d+', '', bible)
shakespeare = re.sub(r'CHAPTER .*', '', shakespeare)
    
shakespeare = text_cleaner(shakespeare[:int(len(shakespeare)/10)])
bible = text_cleaner(bible[:int(len(bible)/400)])

In [187]:
len(bible)

10322

In [188]:
len(shakespeare)

10963

In [189]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
bible_doc = nlp(bible)
shakespeare_doc = nlp(shakespeare)

In [190]:
# Group into sentences.
bible_sents = [[sent, "KJV"] for sent in bible_doc.sents]
shakespeare_sents = [[sent, "Shakespeare"] for sent in shakespeare_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(bible_sents + shakespeare_sents)
sentences.head()

,0,1
0,"(The, Old, Testament, of, the, King, James, Bi...",KJV
1,"(The, First, Book, of, Moses, :, Called, Genesis)",KJV
2,"(In, the, beginning, God, created, the, heaven...",KJV
3,"(And, the, earth, was, without, form, ,, and, ...",KJV
4,"(And, the, Spirit, of, God, moved, upon, the, ...",KJV


In [191]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 50 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
biblewords = bag_of_words(bible_doc)
shakespearewords = bag_of_words(shakespeare_doc)

# Combine bags to create a set of unique words.
common_words = set(biblewords + shakespearewords)

In [192]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 50
Processing row 100
Processing row 150
Processing row 200
Processing row 250
Processing row 300


,heap'd,second,turne,midst,evening,loose,Bible,head,Prima,bring,...,alas,Hast,herb,Calphurnia,light,thought,lose,heauy,text_sentence,text_source
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,"(The, Old, Testament, of, the, King, James, Bi...",KJV
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(The, First, Book, of, Moses, :, Called, Genesis)",KJV
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(In, the, beginning, God, created, the, heaven...",KJV
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(And, the, earth, was, without, form, ,, and, ...",KJV
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(And, the, Spirit, of, God, moved, upon, the, ...",KJV


## Trying out BoW

Now let's give the bag of words features a whirl by trying a random forest.

In [193]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.994475138121547

Test set score: 0.9590163934426229


## BoW with Logistic Regression

Let's try a technique with some protection against overfitting due to extraneous features – logistic regression with ridge regularization (from ridge regression, also called L2 regularization).

In [194]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(penalty='l2') # No need to specify l2 as it's the default. But we put it for demonstration.
train = lr.fit(X_train, y_train)

print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

Training set score: 0.9779005524861878

Test set score: 0.9672131147540983


In [195]:
pd.crosstab(y_train, lr.predict(X_train))

col_0,KJV,Shakespeare
text_source,,
KJV,53,4
Shakespeare,0,124


Logistic regression performs a bit better than the random forest.  

# BoW with Gradient Boosting

And finally, let's see what gradient boosting can do:

In [196]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 1.0

Test set score: 0.9590163934426229


In [200]:
from sklearn.model_selection import cross_val_score

print('Random Forest: ', cross_val_score(rfc, X, Y, cv=10).mean())
print('Logistic Regression: ', cross_val_score(lr, X, Y, cv=10).mean())
print('Gradient Boosting: ', cross_val_score(clf, X, Y, cv=10).mean())

Random Forest:  0.9229588431590656
Logistic Regression:  0.9163922877271042
Gradient Boosting:  0.9128364849833149


Looks like Random Forest is the winner! Let's see if we can improve it.

In [214]:
rfc = ensemble.RandomForestClassifier(criterion='entropy')

train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.994475138121547

Test set score: 0.9672131147540983


In [216]:
print('Random Forest: ', cross_val_score(rfc, X, Y, cv=10).mean())

Random Forest:  0.9128364849833147


In [229]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score
from tqdm import tqdm # show progress bar
import time # time random searches

Y = pd.get_dummies(Y, drop_first=True)

In [249]:
########################################################################
#########    Creating a function in order to quickly call upon    ######
#########         the model with different features  while        ######
#########        cycling through different model parameters       ######
########################################################################

def RandForest_RandomSearch(X, Y, criterion=['gini', 'entropy'],
                                  # max_depth=[3, 4, 5],
                                  # n_estimators=[50, 100, 250],
                                  class_weight=['balanced', 'balanced_subsample', None]
                           ):
    
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

    rfc_hyperparams = dict(
                           criterion = criterion,
                           # max_depth = max_depth,
                           # n_estimators = n_estimators,
                           class_weight = class_weight
                           )

    rfc_hyper_table = list()

    # Timed for loop...start the clock!
    start_time = time.time()
    for trial in tqdm(range(10), position=0):
        params = dict()
        for k, v in rfc_hyperparams.items():
            i = np.random.randint(len(v))
            params[k] = v[i]
        #print(params)
        rfc = ensemble.RandomForestClassifier(**params)
        rfc.fit(X_train, y_train)
        y_pred_train = rfc.predict(X_train)
        y_pred_test = rfc.predict(X_test)
        params['train_precision'] = precision_score(y_train, y_pred_train)
        params['test_precision'] = precision_score(y_test, y_pred_test)
        params['train_recall'] = recall_score(y_train, y_pred_train)
        params['test_recall'] = recall_score(y_test, y_pred_test)
        params['train_acc'] = rfc.score(X_train, y_train)
        params['test_acc'] = rfc.score(X_test, y_test)
        params['train_f1'] = 2 * (params['train_precision'] * params['train_recall'])/(params['train_precision'] + params['train_recall'])
        params['test_f1'] = 2 * (params['test_precision'] * params['test_recall'])/(params['test_precision'] + params['test_recall'])
        params['cross_val_mean'] = cross_val_score(rfc, X, Y, cv=10).mean()
        # Store our parameters and score in a dataframe
        rfc_hyper_table.append(params)

        end_time = time.time()

    print(f'Execution time: {round(end_time - start_time, 2)} seconds')
    rfc_hyper_table = pd.DataFrame(data=rfc_hyper_table)
    return rfc_hyper_table.sort_values(by='cross_val_mean', ascending=False)

In [251]:
# Call the Random Forest function
rand_forest_df = RandForest_RandomSearch(X,Y)
rand_forest_df.style.apply(lambda x: ['background: yellow' if x.name == 'cross_val_mean' else '' for i in x])

100%|██████████| 10/10 [00:03<00:00,  3.16it/s]

Execution time: 3.17 seconds


,criterion,class_weight,train_precision,test_precision,train_recall,test_recall,train_acc,test_acc,train_f1,test_f1,cross_val_mean
2,entropy,balanced_subsample,1,0.944444,1,1,1,0.956044,1,0.971429,0.933626
3,gini,balanced,0.986577,0.971429,1,1,0.990566,0.978022,0.993243,0.985507,0.930515
5,gini,balanced,1,0.944444,1,1,1,0.956044,1,0.971429,0.93007
4,entropy,balanced,0.993243,0.971429,1,1,0.995283,0.978022,0.99661,0.985507,0.923396
7,gini,balanced,1,0.944444,1,1,1,0.956044,1,0.971429,0.920178
6,entropy,None,1,0.944444,1,1,1,0.956044,1,0.971429,0.919841
8,entropy,balanced_subsample,0.993243,0.944444,1,1,0.995283,0.956044,0.99661,0.971429,0.91706
0,entropy,None,0.993243,0.931507,1,1,0.995283,0.945055,0.99661,0.964539,0.91673
9,entropy,None,1,0.944444,1,1,1,0.956044,1,0.971429,0.909711
1,entropy,balanced_subsample,0.993243,0.971429,1,1,0.995283,0.978022,0.99661,0.985507,0.896923


> We've managed to get a slight improvement on our mean cross validation score with consistently high F1 scores by changing the criterion to entropy and using a 'balanced_subsample' class_weight.

+ Results may vary on consecutive execution of the cell.

In [255]:
# Results 
print(rand_forest_df.iloc[0])

criterion                     entropy
class_weight       balanced_subsample
train_precision                     1
test_precision               0.944444
train_recall                        1
test_recall                         1
train_acc                           1
test_acc                     0.956044
train_f1                            1
test_f1                      0.971429
cross_val_mean               0.933626
Name: 2, dtype: object
